# **Predicting RNAcompete binding from RNA bind-n-seq data**

In [ ]:
import pdb
import gdown
import os
from google.colab import drive

## Connect to drive
# Mount your Google Drive to access persistent storage
drive.mount('/content/drive')


Mounted at /content/drive


Import libraries

In [ ]:
import numpy as np
import os
import sys
import keras
from keras.models import Model
import time
import json
from itertools import product
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, GlobalMaxPooling2D, BatchNormalization, Dense, concatenate
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
import pdb
import re
from scipy.stats import pearsonr
from keras.initializers import he_normal
import psutil

General and Common vars

In [ ]:
# Constants
PATH = '/content/drive/MyDrive/data/RBNS_testing/'
VALID_P = 0.2
MAX_SAMPLE_SIZE = 6
FILE_LIMIT = 5000
KERNEL_SIZES = [3, 3, 3]
NUM_OF_KERNELS =  [6, 8, 10]
DENSE_LAYERS = [32]
BATCH_SIZE = 1024
EPOCHS = 2
WORKERS = 1
OUTPUT = True

# Dimension calculation
dim_func = lambda k_size: (MAX_SAMPLE_SIZE + 2 * k_size - 2, 4, 1)

# Create strings for layers and kernels descriptions
layers_description = ''.join(['dense{}_{}'.format(i + 1, dense) for i, dense in enumerate(DENSE_LAYERS)])
kernels_description = ''.join(['kernel{}_{}'.format(size, num) for size, num in zip(KERNEL_SIZES, NUM_OF_KERNELS)])
model_path = 'model_samp{}_{}_{}_batch{}_epoch{}'.format(FILE_LIMIT, kernels_description,
                                                          layers_description,
                                                          BATCH_SIZE, EPOCHS)

# Global variables
class ProjectGlobalVars():
  RBNS_FILES_LIST = []
  OUTPUT_FILE = ''
  INPUT_FILE = ''
  RNCMPT_FILE = ''
  RBNS_FILES_INDEXES = []
  INTENSITY = ''

In [ ]:
class SequenceDataGenerator(keras.utils.Sequence):

    def __init__(self, lines, num_of_classes, kernel_sizes, max_sample_size, batch_size=16, shuffle=False):
        self.kernel_sizes = kernel_sizes
        self.max_sample_size = max_sample_size
        self.dim = [(max_sample_size + 2 * kernel_size - 2, 4, 1) for kernel_size in self.kernel_sizes]
        self.batch_size = batch_size

        self.lines = lines
        self.num_of_classes = num_of_classes

        self.size = len(self.lines)

        self.shuffle = shuffle
        self.indexes = np.arange(self.size)

        self.valid_generator = None

        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.size / self.batch_size))

    def __getitem__(self, index):
        indices = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        x, y = self._data_generation(indices)
        return x, y

    def on_epoch_end(self):
        self.indexes = np.arange(self.size)
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def _data_generation(self, indices_list):
        xs = []
        y = np.empty((self.batch_size, self.num_of_classes))
        for kernel_size, dim in zip(self.kernel_sizes, self.dim):
            x = np.empty((self.batch_size, *dim))
            for i, ind in enumerate(indices_list):
                curr_x = SequenceProcessing.one_hot(self.lines[ind][0], self.max_sample_size, kernel_size)
                x[i, ] = curr_x
            xs.append(x)
        for i, ind in enumerate(indices_list):
          y[i, ] = keras.utils.to_categorical(int(self.lines[ind][1])-1, num_classes=self.num_of_classes)
        return xs, y

class SequenceProcessing(keras.utils.Sequence):
    @staticmethod
    def sliding_window(string, window_size):
      arr = []
      if len(string) <= window_size:
       return [string]
      for i in range(len(string)- window_size + 1):
        arr.append(string[i:i+window_size])
      return arr

    @staticmethod
    def pad(string, max_size):
        string += 'N' * (max_size - len(string))
        return string

    @staticmethod
    def pad_conv(string, kernel_size):
        pad = 'N' * (kernel_size - 1)
        string = pad + string + pad
        return string

    @staticmethod
    def one_hot(string, max_size, kernel_size):
        encoding = {'A': np.array([1, 0, 0, 0]), 'G': np.array([0, 1, 0, 0]),
                    'C': np.array([0, 0, 1, 0]), 'U': np.array([0, 0, 0, 1]),
                    'T': np.array([0, 0, 0, 1]),
                    'N': np.array([0.25] * 4)}

        padded_string = SequenceProcessing.pad(string, max_size)
        vec_list = [encoding[c].reshape(1, -1) for c in SequenceProcessing.pad_conv(padded_string, kernel_size)]

        return np.concatenate(vec_list, axis=0).reshape(len(vec_list), 4, 1)

**Read the files**

In [ ]:
class DataLoader():
  def read_file(path, ind, file_limit):
      lines = []
      count = 0
      with open(path, 'r') as f:
          for line in f:
              seq = line.strip().split()[0]
              lines.append((seq, ind))
              count += 1
              if file_limit and count >= file_limit:
                  break
      return lines

  def read_files(rbns_files, file_limit=None):
      lines_from_all_files = []
      for ind, file in enumerate(rbns_files):
          lines_from_all_files += DataLoader.read_file(file, ind, file_limit)
      return lines_from_all_files

  def read_file_rncmpt(file_path):
    sequences = []
    with open(file_path) as f:
        for line in f:
            sequences.append(line.strip())
    return sequences


  def read_file_rncmpt_intensity(path):
    intensity = []
    with open(path) as f:
        for line in f:
            intensity.append(line.strip())
    return intensity

  def get_sorted_files(file_paths):
    lst = []
    for file in file_paths:
        concentration = file.split('_')[-1].split('nM')[0]
        concentration_val = 0
        if concentration not in 'input.seq':
            concentration_val = int(concentration)
        lst.append((file, concentration_val))
    lst.sort(key=lambda x: x[1])
    return lst

  def get_files_list(rbp_ind):
      substr = 'RBP' + str(rbp_ind) + '_'
      if ProjectGlobalVars.RBNS_FILES_LIST:
        files = ProjectGlobalVars.RBNS_FILES_LIST
      else:
        files = [PATH+file for file in os.listdir(PATH) if substr in file]
      lst = DataLoader.get_sorted_files(files)
      return lst

  def load_files(rbp_num):
      cmpt_path = ProjectGlobalVars.RNCMPT_FILE
      files = DataLoader.get_files_list(rbp_num)
      cmpt_file = DataLoader.read_file_rncmpt(cmpt_path)
      return files, cmpt_file, rbp_num

In [ ]:
def get_training_sixes(lines_from_all_files):
  dic = {}
  sub_rnas = []
  for (line, ind) in lines_from_all_files:
    for sub_rna in SequenceProcessing.sliding_window(line, 6):
       sub_rnas.append((sub_rna, ind))
  return sub_rnas

def create_train_valid_data(files, num_of_classes, kernel_size=None,
                            custom_file_limit=None):
    """Create generators for train an validation"""

    lines_from_all_files = DataLoader.read_files(files, file_limit=custom_file_limit)
    sub_rnas = get_training_sixes(lines_from_all_files)

    np.random.shuffle(sub_rnas)

    valid_n = int(VALID_P * len(sub_rnas))

    validation_data = sub_rnas[:valid_n]
    train_data = sub_rnas[valid_n:]
    train_gen = SequenceDataGenerator(train_data, num_of_classes=num_of_classes, kernel_sizes=kernel_size,
                              max_sample_size=MAX_SAMPLE_SIZE,
                              batch_size=BATCH_SIZE, shuffle=True)

    valid_gen = None
    if valid_n > 0:
        valid_gen = SequenceDataGenerator(validation_data, num_of_classes=num_of_classes, kernel_sizes=kernel_size,
                                  max_sample_size=MAX_SAMPLE_SIZE,
                                  batch_size=BATCH_SIZE, shuffle=True)
    return train_gen, valid_gen

def create_model(num_classes, dense_layers=None, kernel_sizes=None, num_of_kernels=None, dropout_rate=0.5):
    """Model with different kernel sizes"""

     # Create input layers for each specified kernel size
    inputs = [Input(shape=dim_func(kernel_size)) for kernel_size in kernel_sizes]
    kernels = []

    # Loop through each kernel size and corresponding number of kernels
    for k_size, num_of_kernel, inp in zip(kernel_sizes, num_of_kernels, inputs):
        conv_layer = Conv2D(num_of_kernel, (k_size, 4), strides=(1, 1),
                            padding='valid', kernel_initializer=he_normal())(inp)
        activation_layer = Activation('relu')(conv_layer)

        max_pooling_layer = GlobalMaxPooling2D()(activation_layer)  # Add MaxPooling2D after Conv2D
        kernels.append(max_pooling_layer)

    kernels = [BatchNormalization()(kernel) for kernel in kernels]

    # Concatenate the normalized kernel outputs along axis 1
    func = concatenate(kernels, axis=1)

    for dense_size in dense_layers:
        func = Dense(dense_size, activation='relu')(func)

    # Add the final dense layer with sigmoid activation for output
    func = Dense(num_classes, activation='sigmoid')(func)

    model = Model(inputs, func)
    model.summary()
    return model

def create_and_compile_model(num_of_classes, loss_func, dense_layers=None, kernel_size=None,
                             num_of_kernels=None):
    model = create_model(num_of_classes, dense_layers=dense_layers, kernel_sizes=kernel_size,
                         num_of_kernels=num_of_kernels)
    opt = keras.optimizers.Adam(learning_rate=0.01)

    model.compile(loss=loss_func,
                  optimizer=opt,
                  metrics=['accuracy'])
    return model

def train_model(model, train_gen, valid_gen, epochs, use_multiprocessing=False):
    model.fit(x=train_gen,
              validation_data=valid_gen,
              epochs=epochs,
              steps_per_epoch=len(train_gen),
              validation_steps=len(valid_gen),
              use_multiprocessing=use_multiprocessing,
              workers=WORKERS)
    return model

def train_pipeline(files, dense_layer, kernel_size, num_of_kernel, c_file_limit, epochs, rbp_ind):
    """The function creates a model, datasets and train model. Then, return the trained model"""
    num_of_classes = len(DataLoader.get_files_list(rbp_ind))
    loss_func = 'categorical_crossentropy'

    model = create_and_compile_model(num_of_classes, loss_func, dense_layers=dense_layer,
                                     kernel_size=kernel_size, num_of_kernels=num_of_kernel)

    train_gen, valid_gen = create_train_valid_data(files=files,
                                                   num_of_classes=num_of_classes,
                                                   kernel_size=kernel_size,
                                                   custom_file_limit=c_file_limit)

    model = train_model(model, train_gen, valid_gen, epochs)
    return model

def calc_pearson(y_pred_scores):
    trueValue = [float(i) for i in ProjectGlobalVars.INTENSITY]
    correlation, _ = pearsonr(trueValue, y_pred_scores)

    return y_pred_scores, correlation

def predict(model, kernel_size, cmpt_seqs, rbp_num):
    """Use model to predict אhe test dataset, and create a sorted RNCMPT file"""

    # Set a map for all possible 6-letter sub-rna's and their prediction, to save prediction time.
    letters = ['A', 'G', 'C', 'U']
    window_length = 6

    # Generate all possible combinations of 6-letter sequences
    combinations = product(letters, repeat=window_length)
    result = [''.join(combination) for combination in combinations]

    # Prepare the input data for prediction
    x_test = []
    for kernel_size_i in kernel_size:
        x_curr = np.array([SequenceProcessing.one_hot(r, MAX_SAMPLE_SIZE, kernel_size_i) for r in result])
        x_test.append(x_curr)
    y_pred_model = model.predict(x_test)

    # Create a dictionary mapping sequences to their predicted values
    dict_x_y = {key: value for key, value in zip(result, y_pred_model)}

    """
    Go over each RNA using sliding window at size 6.
    For the results of all the sixes of each RNA, save the max value of highest
    concentration, the max value of second highest concentration and
    the minimum value of the lowest concentration.
    Then create aggregation function:
    Max(highest) + Max(2nd highest) - Min(lowest)
    Max[-1]      + Max[-2]          - Min[0]
    and this is our binding score.
    """
    y_pred_scores = []

    for i, cmpt in enumerate(cmpt_seqs):
        max_cycle_last = 0
        max_cycle_next_last = 0
        min_cycle_first = 999
        intensity_pred = 0
        # Slide a window of size 6 over the RNA sequence
        for seq in SequenceProcessing.sliding_window(cmpt, 6):
            x_test = []
            y_pred = dict_x_y[seq]

            # Update values for max and min predictions
            if y_pred[0] < min_cycle_first:
                min_cycle_first = y_pred[0]
            if y_pred[-1] > max_cycle_last:
                max_cycle_last = y_pred[-1]
            if y_pred[-2] > max_cycle_next_last:
                max_cycle_next_last = y_pred[-2]

        # Calculate the binding score for the RNA sequence
        intensity_pred = float(max_cycle_last) + float(max_cycle_next_last) - float(min_cycle_first)
        print(i, " ", intensity_pred)
        y_pred_scores.append(intensity_pred)

    # Output the calculated scores to a file
    output_rncmpt(y_pred_scores, rbp_num)
    pearson = 0
    scores = 0

    # TODO: Calculate Pearson correlation if INTENSITY is available
    # if "--projectAll" in sys.argv:
    #     scores, pearson = calc_pearson(y_pred_scores)
    #     print("pearson", pearson)

    return scores, pearson

**Analyse results**

In [ ]:
def write_pearson(pearson, rbp_num):
    with open('/content/drive/MyDrive/data/Results/RBP_{}.txt'.format(rbp_num), 'w') as f:
        f.write(str(pearson))

def output_rncmpt(scores, rbp_num):
    outputFile = '/content/drive/MyDrive/data/RNCMPT_training_results/RBP{}.txt'.format(rbp_num)
    if ProjectGlobalVars.OUTPUT_FILE:
      outputFile = ProjectGlobalVars.OUTPUT_FILE

    with open(outputFile, 'w') as f:
        for score in scores:
            f.write(str(score) + '\n')

The following function is called from the main method. The function start the DL process

***Init DL training***

In [ ]:
def set_global_variables():
  if "--projectAll" in sys.argv:
      # In case there is a project flag, should set all the training and testing data, and RNCMPT_tarining as well.
      ProjectGlobalVars.RNCMPT_FILE = "/content/drive/MyDrive/data/RNAcompete_sequences.txt"
      # Itraration and check what are the RMNS files are
      ProjectGlobalVars.RBNS_FILES_INDEXES = [i for i in range(17, 18)]
  else:
      # TODO: delete before sending the ex.
      # sys.argv = []
      # sys.argv.append("temp")
      # sys.argv.append("/content/drive/MyDrive/data/RNCMPT_training_results/RBP1.txt")
      # sys.argv.append("/content/drive/MyDrive/data/RNAcompete_sequences.txt")
      # sys.argv.append("/content/drive/MyDrive/data/RBNS_training/RBP7_input.seq")
      # sys.argv.append("/content/drive/MyDrive/data/RBNS_training/RBP7_20nM.seq /content/drive/MyDrive/data/RBNS_training/RBP7_80nM.seq /content/drive/MyDrive/data/RBNS_training/RBP7_1300nM.seq")

      # Check command flags and args
      if len(sys.argv) < 4:
           raise BaseException("Number of arguments should be at least 4")


      # An executation example:
      # python main.py <ofile> <RNCMPT> <input> <RBNS1> <RBNS2> … <RBNS5>
      # ofile - outputfile
      # The RBNS files are given by increasing concentrations (i.e., 5nM, 80nM, 1300nM)
      ProjectGlobalVars.OUTPUT_FILE = sys.argv[1]
      ProjectGlobalVars.RNCMPT_FILE = sys.argv[2]
      ProjectGlobalVars.INPUT_FILE = sys.argv[3]
      files = sys.argv[4:]
      ProjectGlobalVars.RBNS_FILES_LIST = files[0].split()
      ProjectGlobalVars.RBNS_FILES_INDEXES = []

      # Extract numbers after "RBP" for each element in the input list
      rbp_numbers = [int(item.split("RBP")[1].split("_")[0]) for item in ProjectGlobalVars.RBNS_FILES_LIST if "RBP" in item]
      ProjectGlobalVars.RBNS_FILES_INDEXES = []
      ProjectGlobalVars.RBNS_FILES_INDEXES.append(rbp_numbers[0])


def main():
  set_global_variables()
  for rbp_index in ProjectGlobalVars.RBNS_FILES_INDEXES:
      print('Starting deep learning proccess')
      start = time.time()

      files, cmpt_seqs, rbp_num = DataLoader.load_files(rbp_index)

      # if "--projectAll" in sys.argv:
      #   ProjectGlobalVars.INTENSITY = DataLoader.read_file_rncmpt_intensity("/content/drive/MyDrive/data/RNCMPT_training/RBP{}.txt".format(rbp_num))

      training_files = [i[0] for i in files]

      # Save model to prevent creating the model and train it again
      model = train_pipeline(training_files, DENSE_LAYERS, KERNEL_SIZES, NUM_OF_KERNELS,
                                  FILE_LIMIT, EPOCHS, rbp_num)

      scores, pearson = predict(model, KERNEL_SIZES, cmpt_seqs, rbp_num)

      write_pearson(pearson, rbp_num)

      end = time.time()
      print('Total time', (end - start) / 60, 'minutes')

      process = psutil.Process()
      memory_usage = process.memory_info().rss / (1024 * 1024)  # in MB
      print("Memory usage:", memory_usage, "MB")

      cpu_usage = psutil.cpu_percent(interval=1)  # 1 second interval
      print("CPU usage:", cpu_usage, "%")



In [ ]:
if __name__ == "__main__":

    # Add flag when running all training data
    # sys.argv.extend(["--projectAll"])

    main()

Streaming output truncated to the last 5000 lines.
236360   0.6401014626026154
236361   0.6491593420505524
236362   0.7640016078948975
236363   0.6464997828006744
236364   0.6507667601108551
236365   0.8017900586128235
236366   0.7234609723091125
236367   0.6736453473567963
236368   0.6362007558345795
236369   0.6236491799354553
236370   0.6236491799354553
236371   0.6332210600376129
236372   0.7134186923503876
236373   0.6348741948604584
236374   0.6063462495803833
236375   0.6763380169868469
236376   0.6701584458351135
236377   0.6015533804893494
236378   0.5941360592842102
236379   0.6077073514461517
236380   0.6131839156150818
236381   0.610643744468689
236382   0.6485766768455505
236383   0.6294445693492889
236384   0.6703527569770813
236385   0.6925926506519318
236386   0.6396113038063049
236387   0.6222379505634308
236388   0.6788018643856049
236389   0.6067655682563782
236390   0.6757219135761261
236391   0.6693196892738342
236392   0.6667128801345825
236393   0.591574370861053